# morphofit example usage notebook

In this notebook, we provide examples on how to run morphofit on a set of two simulated HST images mimicking the MACS J1149.5+2223 observational conditions in the Frontier Fields survey in the F814W and F160W wavebands.

morphofit is designed to be ran via command-line using esub-epipe. It can also be run in a notebook either by calling the scripts in the `main_functions` folder with `subprocess` or by calling the individual functions or the main function of the scripts in the notebook cells.

In order to run it, the user should create a folder named as the `target_name`, e.g. macs1149. All initial and produced data are going to be stored in this folder. The user should also rename images following the naming convention:

`telescopename_targetname_waveband_imagetype.fits`

e.g. HST_macs1149_f814_drz.fit

See the folder `demo_data` for an example.

__The example commands on how to run the morphofit modules in the terminal are stored in the `pipeline.yaml` file in the `main_functions` folder__

In [ ]:
import subprocess
import os
from astropy.table import Table, Column
import numpy as np

### How to use morphofit to run SExtractor in forced photometry mode on all provided images in all wavebands

SExtractor does not come pre-packaged with _morphofit_, but it needs to be installed separately.

One way to do this is by using _brew_ https://formulae.brew.sh/formula/sextractor

In [ ]:
mode = 'run-tasks' # whether to run the script in serial (run) or in parallel (run-tasks, run-mpi)
tasks = '0>1' # total number of jobs to run, e.g. '0>3' is the syntax for running SExtractor on
              # three different galaxy clusters. Since we have one cluster example, tasks is just 0>1
              # in the case of galfit fit on stamps, the number of tasks is given by 
              # n_target_galaxies * n_bands * n_combinations_background_sigmaimage_psf
root_path = './demo_data/'# Path to root folder where data are stored and where code is run
wavebands_list = 'f814w,f160w' # string of comma-separated waveband names
h5pytable_folder = './demo_data/h5table/' # path to folder where the hdf5 table with parameters relevant for 
                                          # morphofit are stored
h5pytable_prefix = 'sextractor_run_table' # filename prefix of hdf5 table storing parameters relevant for morphofit 
telescope_name = 'HST' # name of the telescope/instrument/survey data used
target_field_names = 'macs1149' # name of the target to analyse, it has to be the same as targetname
sci_images_suffix = 'drz.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the science image
rms_images_suffix = 'rms.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the root mean square image, if unavailable set it to 'None'
exp_images_suffix = 'exp.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the exposure time image, if unavailable set it to 'None'
init_ext_star_cat_suffix = 'star_positions.fits' # suffix of the fits table storing the star positions in the image
image_archive_prefix = 'images' # filename prefix of the generated tar file containing images
resources_archive_prefix = 'res_sextractor_files' # filename prefix of the generated tar file containing res files
star_catalogues_path = './demo_data/star_catalogues/' # path to star catalogues folder, a first guess for the star catalogue should be provided by the user
pixel_scale = 0.060 # pixel scale in arcsec/pixel
psf_fwhm_init_guesses = '0.1,0.1' # initial guesses for the seeing FWHM in arcsec, e.g. 0.1 arcsec for HST
sextractor_binary_filename = '/opt/homebrew/bin/sex' # path to SExtractor executable
# from detect_minarea to sextractor_checkimages_endings, the parameters have the same meaning
# of the corresponding SExtractor ones
sextractor_resources_path = '../morphofit/res/sextractor/' # path to folder storing SExtractor required files
function = 'all' # it can be all, main, check_missing and merge
n_cores = os.cpu_count() # number of available CPU for parallel computing

subprocess.run(['esub', '../main_functions/create_table_for_sextractor.py', '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--wavebands_list={}'.format(wavebands_list),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--telescope_name={}'.format(telescope_name),
                '--target_field_names={}'.format(target_field_names),
                '--sci_images_suffix={}'.format(sci_images_suffix),
                '--rms_images_suffix={}'.format(rms_images_suffix),
                '--exp_images_suffix={}'.format(exp_images_suffix),
                '--ext_star_cat_suffix={}'.format(init_ext_star_cat_suffix),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--star_catalogues_path={}'.format(star_catalogues_path),
                '--pixel_scale={}'.format(pixel_scale), '--psf_fwhm_init_guesses={}'.format(psf_fwhm_init_guesses),
                '--detect_minarea=10', '--detect_thresh=1.0', '--analysis_thresh=1.5', '--deblend_nthresh=64',
                '--deblend_mincont=0.0001', '--phot_apertures=3,5,8,10,13,15,18,20,23,25,28,30',
                '--phot_autoparams=2.5,3.5', '--phot_petroparams=2.0,3.5', '--phot_autoapers=0.0,0.0',
                '--phot_fluxfrac=0.5', '--back_size=64', '--back_filtersize=3',
                '--sextractor_binary_filename={}'.format(sextractor_binary_filename),
                '--sextractor_config_filename=default.sex', '--sextractor_params_filename=default.param',
                '--sextractor_filter_filename=gauss_3.0_5x5.conv', '--sextractor_nnw_filename=default.nnw',
                '--sextractor_checkimages=SEGMENTATION', '--sextractor_checkimages_endings=seg',
                '--sextractor_resources_path={}'.format(sextractor_resources_path),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])


In [ ]:
temp_dir_path = './demo_data/' # temporary folders location where morphofit single core jobs are run
local_or_cluster = 'local' # it can be either local or cluster, set it to local for own machine use, cluster for facilities using Slurm as scheduler
sextractor_forced_catalogue_suffix = 'forced.cat' # user-defined suffix for the single-band output SExtractor catalogue
sextractor_ra_keyword = 'ALPHAWIN_J2000' # right ascension column name to be used to match galaxies across bands
sextractor_dec_keyword = 'DELTAWIN_J2000' # declination column name to be used to match galaxies across bands
star_catalogue_ra_keyword = 'ALPHAWIN_J2000' # right ascension column name to be used to find stars in images and measure 
                                 # their profile FWHM
star_catalogue_dec_keyword = 'DELTAWIN_J2000' # declination column name to be used to find stars in images and measure 
                                   # their profile FWHM
detection_image_suffix = 'detection.fits' # suffix of the generated detection image
parameters_table_suffix = 'param_table.fits' # suffix of the fits table storing obtained instrumental and 
                                             # observational parameters
multiband_catalogue_suffix = 'multiband.forced.cat' # user-defined suffix for the multiband output SExtractor catalogue

subprocess.run(['esub', '../main_functions/main_sextractor.py', '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--temp_dir_path={}'.format(temp_dir_path),
                '--local_or_cluster={}'.format(local_or_cluster),
                '--sextractor_forced_catalogue_suffix={}'.format(sextractor_forced_catalogue_suffix),
                '--sextractor_checkimages_endings=seg',
                '--sextractor_ra_keyword={}'.format(sextractor_ra_keyword),
                '--sextractor_dec_keyword={}'.format(sextractor_dec_keyword),
                '--star_catalogue_ra_keyword={}'.format(star_catalogue_ra_keyword),
                '--star_catalogue_dec_keyword={}'.format(star_catalogue_dec_keyword),
                '--detection_image_suffix={}'.format(detection_image_suffix),
                '--parameters_table_suffix={}'.format(parameters_table_suffix),
                '--multiband_catalogue_suffix={}'.format(multiband_catalogue_suffix),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

### How to use morphofit to create PSF images

Before creating the PSF images for GALFIT, the user needs to generate a star catalogue for each waveband that needs to be fitted. One way to do this is by exploiting the _HST_macs1149_multiband.forced.cat_ to select high signal-to-noise, non-saturated, isolated stars.

In [ ]:
mode = 'run' # whether to run the script in serial (run) or in parallel (run-tasks, run-mpi)
tasks = '0>1'
root_path = './demo_data/'
h5pytable_folder = './demo_data/h5table/'
h5pytable_prefix = 'table_psf_creation'
wavebands_list = 'f814w,f160w'
target_field_names = 'macs1149'
parameters_table_suffix = 'param_table.fits' # suffix of the table containing the image parameters measured in the previous module
sci_images_suffix = 'drz.fits'
image_archive_prefix = 'images_for_psf' # filename prefix of the generated tar file containing images
resources_archive_prefix = 'res_psf_files' # filename prefix of the generated tar file containing res files
ext_star_cat_suffix = 'star_positions.fits' # suffix of the fits table storing the star positions in the image
star_catalogues_path = './demo_data/star_catalogues/' # path to star catalogues folder
psf_image_size = 50 # the psf image is assumed to be squared
pixel_scale = 0.060 # pixel scale in arcsec/pixel
function = 'all'
n_cores = 1

subprocess.run(['esub', '../main_functions/create_table_for_psf_images_per_target_field.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--wavebands_list={}'.format(wavebands_list),
                '--target_field_names={}'.format(target_field_names),
                '--parameters_table_suffix={}'.format(parameters_table_suffix),
                '--sci_images_suffix={}'.format(sci_images_suffix),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--ext_star_cat_suffix={}'.format(ext_star_cat_suffix),
                '--star_catalogues_path={}'.format(star_catalogues_path),
                '--psf_image_size={}'.format(psf_image_size),
                '--pixel_scale={}'.format(pixel_scale),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

In [ ]:
mode = 'run-tasks' # whether to run the script in serial (run) or in parallel (run-tasks, run-mpi)
tasks = '0>1' # total number of jobs to run, e.g. '0>3' is the syntax for running the PSF estimation on
              # three different galaxy clusters. Since we have one cluster example, tasks is just 0>1
h5pytable_folder = './demo_data/h5table/'
h5pytable_prefix = 'table_psf_creation'
temp_dir_path = './demo_data/'
image_archive_prefix = 'images_for_psf' # filename prefix of the generated tar file containing images
resources_archive_prefix = 'res_psf_files' # filename prefix of the generated tar file containing res files
local_or_cluster = 'local'
psf_methods = 'moffat,observed,pca' # types of psf estimation methods, available are moffat,observed,pca,effective(HST only)
star_catalogue_x_keyword = 'XWIN_IMAGE_f814w' # x keyword in the star catalogue containing the stars pixel positions
star_catalogue_y_keyword = 'YWIN_IMAGE_f814w' # y keyword in the star catalogue containing the stars pixel positions
function = 'all'
n_cores = 1

subprocess.run(['esub', '../main_functions/main_create_psf_images_per_target_field.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--temp_dir_path={}'.format(temp_dir_path),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--local_or_cluster={}'.format(local_or_cluster),
                '--psf_methods={}'.format(psf_methods),
                '--star_catalogue_x_keyword={}'.format(star_catalogue_x_keyword),
                '--star_catalogue_y_keyword={}'.format(star_catalogue_y_keyword),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

### How to use morphofit to run GALFIT on stamps

GALFIT does not come pre-packaged with _morphofit_, but it needs to be downloaded from https://users.obs.carnegiescience.edu/peng/work/galfit/galfit.html

This part requires a target catalogue and a sources catalogue. The former contains the information about the relevant sources for which we want to estimate the structural parameters. The latter contains all the other sources such that neighbouring objects are fitted simultaneously.

One possible selection could be to define as targets all galaxies having MAG_AUTO_f814<=22.5, while defining as sources all galaxies having 22.5<MAG_AUTO_f814w<27.

In [ ]:
root = './demo_data/macs1149/'
m1149_se_cat = Table.read(os.path.join(root, 'HST_macs1149_multiband.forced.cat'))
target_mask = np.where((m1149_se_cat['MAG_AUTO_f814w']<=22.5)&(m1149_se_cat['CLASS_STAR_f814w']<0.95))
sources_mask = np.where((m1149_se_cat['MAG_AUTO_f814w']>22.5)&(m1149_se_cat['MAG_AUTO_f814w']<27)&
                        (m1149_se_cat['CLASS_STAR_f814w']<0.95))
m1149_se_cat[target_mask][2:3].write(os.path.join(root, 'HST_macs1149_multiband_targets.cat'), format='fits',
                                overwrite=True) # we select only one object for the sake of speed
m1149_se_cat[sources_mask].write(os.path.join(root, 'HST_macs1149_multiband_sources.cat'), format='fits',
                                overwrite=True)

Once the two catalogues have been created by the user, the columns 'SERSIC_INDEX', 'TOFIT', 'LIGHT_PROFILE' and 'COMPONENT_NUMBER' needs to be appended to both catalogues:
- *SERSIC_INDEX_{}*.format(waveband), initial values of the Sersic indices for the fit with GALFIT

- *TOFIT_{}*.format(waveband),  whether we want to fit that source or keep its values fixed

- *LIGHT_PROFILE_{}*.format(waveband),  the type of light profile we would like to fit (sersic,devauc,expdisk)

- *COMPONENT_NUMBER*, the integer index denoting the specific component we are fitting to the source. If, e.g. we do a bulge plus disk decomposition, each source will have two rows in the table, one with COMPONENT_NUMBER=0 and LIGHT_PROFILE=devauc, and the other with COMPONENT_NUMBER=1 and LIGHT_PROFILE=expdisk. 

In [ ]:
root = './demo_data/macs1149/'
target_cat = Table.read(os.path.join(root, 'HST_macs1149_multiband_targets.cat'), format='fits')
source_cat = Table.read(os.path.join(root, 'HST_macs1149_multiband_sources.cat'), format='fits')

wavebands = ['f814w', 'f160w']
for waveband in wavebands:
    c = Column(np.full(len(target_cat), 2.5), name='SERSIC_INDEX_{}'.format(waveband))
    target_cat.add_column(c)
    c = Column(np.full(len(source_cat), 2.5), name='SERSIC_INDEX_{}'.format(waveband))
    source_cat.add_column(c)
    c = Column(np.full(len(target_cat), 1, dtype=int), name='TOFIT_{}'.format(waveband))
    target_cat.add_column(c)
    c = Column(np.full(len(source_cat), 1, dtype=int), name='TOFIT_{}'.format(waveband))
    source_cat.add_column(c)
    c = Column(np.full(len(target_cat), 'sersic', dtype='U6'), name='LIGHT_PROFILE_{}'.format(waveband))
    target_cat.add_column(c)
    c = Column(np.full(len(source_cat), 'sersic', dtype='U6'), name='LIGHT_PROFILE_{}'.format(waveband))
    source_cat.add_column(c)
    
c = Column(np.full(len(target_cat), 0, dtype=int), name='COMPONENT_NUMBER')
target_cat.add_column(c)
c = Column(np.full(len(source_cat), 0, dtype=int), name='COMPONENT_NUMBER')
source_cat.add_column(c)
    
target_cat.write(os.path.join(root, 'HST_macs1149_multiband_targets.cat'), format='fits', overwrite=True)
source_cat.write(os.path.join(root, 'HST_macs1149_multiband_sources.cat'), format='fits', overwrite=True)


In [ ]:
mode = 'run' 
tasks = '0>1'
root_path = './demo_data/'
h5pytable_folder = './demo_data/h5table/'
h5pytable_filename = 'table_galfit_on_stamps_run.h5'
telescope_name = 'HST'
target_field_names = 'macs1149'
wavebands_list = 'f814w,f160w'
psf_image_types_list='moffat_psf,observed_psf,pca_psf' # types of psf estimation methods, available are moffat,observed,pca,effective(HST only)
sigma_image_types_list='custom_sigma_image,internal_generated_sigma_image' # types of sigma image generation methods, available are custom_sigma_image,internal_generated_sigma_image
background_estimate_methods_list='background_free_fit,background_fixed_value' # types of background estimation methods, available are background_free_fit,background_fixed_value
enlarging_image_factor=20 # factor by which the stamp around the galaxy is enlarged as function of the target galaxy effective radius 
enlarging_separation_factor=10 # if a neighbouring galaxy is closer to the target than this factor times the target galaxy effective radius, then the neighbouring galaxy is simultaneously fit
pixel_scale = 0.060
convolution_box_size=256
galfit_binary_file='/usr/local/bin/galfit' # user-dependent
target_galaxies_catalogue_suffix='multiband_targets.cat'
target_galaxies_id_key='NUMBER'
source_galaxies_catalogue_suffix='multiband_sources.cat'
parameters_table_suffix = 'param_table.fits'
sci_images_suffix = 'drz.fits'
rms_images_suffix = 'rms.fits' 
exp_images_suffix = 'exp.fits'
seg_images_suffix = 'drz.forced_seg.fits'
function = 'main'
n_cores = 1

subprocess.run(['esub', '../main_functions/create_table_for_galfit_stamps.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_filename={}'.format(h5pytable_filename),
                '--telescope_name={}'.format(telescope_name),
                '--target_field_names={}'.format(target_field_names),
                '--wavebands_list={}'.format(wavebands_list),
                '--psf_image_types_list={}'.format(psf_image_types_list),
                '--sigma_image_types_list={}'.format(sigma_image_types_list),
                '--background_estimate_methods_list={}'.format(background_estimate_methods_list),
                '--enlarging_image_factor={}'.format(enlarging_image_factor),
                '--enlarging_separation_factor={}'.format(enlarging_separation_factor),
                '--pixel_scale={}'.format(pixel_scale),
                '--convolution_box_size={}'.format(convolution_box_size),
                '--galfit_binary_file={}'.format(galfit_binary_file),
                '--target_galaxies_catalogue_suffix={}'.format(target_galaxies_catalogue_suffix),
                '--target_galaxies_id_key={}'.format(target_galaxies_id_key),
                '--source_galaxies_catalogue_suffix={}'.format(source_galaxies_catalogue_suffix),
                '--parameters_table_suffix={}'.format(parameters_table_suffix),
                '--sci_images_suffix={}'.format(sci_images_suffix),
                '--rms_images_suffix={}'.format(rms_images_suffix),
                '--exp_images_suffix={}'.format(exp_images_suffix),
                '--seg_images_suffix={}'.format(seg_images_suffix),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

temporary directory are removed only if the code runs successfully. If there is any error, the user should manually delete the temporary directory.

The results of the fit are saved in sub-directories of the stamps directory in ./demo_data/macs1149

In [ ]:
mode = 'run-tasks' 
tasks = '0>24' # number of tasks is equal to the number of combinations from the previous cell
h5pytable_folder = './demo_data/h5table/'
h5pytable_filename = 'table_galfit_on_stamps_run.h5'
temp_dir_path = '/Users/Luca.Tortorelli/morphofit/examples/demo_data/' # here we always need an absolute path
files_archive_prefix='galfit_res'
target_galaxies_keys='NUMBER,XWIN_IMAGE_f814w,YWIN_IMAGE_f814w,ALPHAWIN_J2000_f814w,DELTAWIN_J2000_f814w,MAG_AUTO,FLUX_RADIUS,FLUX_RADIUS_f814w,SERSIC_INDEX,BWIN_IMAGE,AWIN_IMAGE,THETAWIN_SKY,TOFIT,COMPONENT_NUMBER,LIGHT_PROFILE' # keyword names in the targets catalogue for the properties needed by morphofit
source_galaxies_keys='NUMBER,ALPHAWIN_J2000_f814w,DELTAWIN_J2000_f814w,MAG_AUTO,FLUX_RADIUS,SERSIC_INDEX,BWIN_IMAGE,AWIN_IMAGE,THETAWIN_SKY,TOFIT,COMPONENT_NUMBER,LIGHT_PROFILE' # keyword names in the sources catalogue for the properties needed by morphofit
display_type_galfit='regular'
galfit_options='0'
diagnostic_plots='True' # True to save diagnostic plots, False otherwise
phot_apertures='3,5,8,10,13,15,18,20,23,25,28,30' # same as those defined when running SExtractor
local_or_cluster='local'
function = 'main'
n_cores = os.cpu_count()

subprocess.run(['esub', '../main_functions/main_galfit_stamps.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_filename={}'.format(h5pytable_filename),
                '--temp_dir_path={}'.format(temp_dir_path),
                '--files_archive_prefix={}'.format(files_archive_prefix),
                '--target_galaxies_keys={}'.format(target_galaxies_keys),
                '--source_galaxies_keys={}'.format(source_galaxies_keys),
                '--display_type_galfit={}'.format(display_type_galfit),
                '--galfit_options={}'.format(galfit_options),
                '--diagnostic_plots={}'.format(diagnostic_plots),
                '--phot_apertures={}'.format(phot_apertures),
                '--local_or_cluster={}'.format(local_or_cluster),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

### How to combine the results from multiple fit on stamps in a single catalogue

These morphofit module creates a single multiband catalogue that contains both the SExtractor and the GALFIT results. Each galaxy has best-fitting parameters from different combinations of psf, sigma image and background. This module combines them into a single weighted average estimate.

In [ ]:
mode = 'run' 
tasks = '0>1'
root_path = './demo_data/'
h5pytable_folder = './demo_data/h5table/'
h5pytable_filename = 'table_masterstamps.h5'
telescope_name = 'HST'
target_field_names = 'macs1149'
wavebands_list = 'f814w,f160w'
psf_image_types_list='moffat_psf,observed_psf,pca_psf'
sigma_image_types_list='custom_sigma_image,internal_generated_sigma_image'
background_estimate_methods_list='background_free_fit,background_fixed_value'
target_galaxies_catalogue_suffix='multiband_targets.cat'
source_galaxies_catalogue_suffix='multiband_sources.cat'
function = 'all'
n_cores = 1

subprocess.run(['esub', '../main_functions/create_table_for_masterstamps.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_filename={}'.format(h5pytable_filename),
                '--telescope_name={}'.format(telescope_name),
                '--target_field_names={}'.format(target_field_names),
                '--wavebands_list={}'.format(wavebands_list),
                '--psf_image_types_list={}'.format(psf_image_types_list),
                '--sigma_image_types_list={}'.format(sigma_image_types_list),
                '--background_estimate_methods_list={}'.format(background_estimate_methods_list),
                '--target_galaxies_catalogue_suffix={}'.format(target_galaxies_catalogue_suffix),
                '--source_galaxies_catalogue_suffix={}'.format(source_galaxies_catalogue_suffix),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

The results of the combination is saved in ./demo_data/macs1149

In [ ]:
mode = 'run-tasks' 
tasks = '0>1' # number of tasks is equal to the number of target fields, in this case the cluster m1149
h5pytable_folder = './demo_data/h5table/'
h5pytable_filename = 'table_masterstamps.h5'
temp_dir_path = './demo_data/'
galfit_properties_mastertable_suffix='mastertable.fits'
source_galaxies_catalogue_id_key = 'NUMBER'
galaxy_ids_key = 'NUMBER'
light_profiles_key = 'LIGHT_PROFILE'
galaxy_components_key = 'COMPONENT_NUMBER'
function = 'all'
n_cores = 1

subprocess.run(['esub', '../main_functions/main_create_masterstamps_catalogue.py',
                '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_filename={}'.format(h5pytable_filename),
                '--temp_dir_path={}'.format(temp_dir_path),
                '--galfit_properties_mastertable_suffix={}'.format(galfit_properties_mastertable_suffix),
                '--source_galaxies_catalogue_id_key={}'.format(source_galaxies_catalogue_id_key),
                '--galaxy_ids_key={}'.format(galaxy_ids_key),
                '--light_profiles_key={}'.format(light_profiles_key),
                '--galaxy_components_key={}'.format(galaxy_components_key),
                '--local_or_cluster={}'.format(local_or_cluster),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])

Similar steps apply to the fit in regions and on the full image.